In [1]:
import torch
import torch.nn.functional as F 
import torch.utils.data as data
import torch.backends.cudnn as cudnn
import torch.optim as optim

import csv
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [2]:
'''
data processing
'''
BATCH_SIZE = 256
class FaceLandmarksDataset(data.Dataset):
    """Face Landmarks dataset."""
    def __init__(self, csv_file,datatype='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file, iterator=True)
        self.datatype = datatype
        self.csv_file = csv_file
        self.epoch = 1
        self.trainNum = 26518
        self.testNum = 5303
        
        #Todo 
#         self.landmarks_frame[] # set trainset (0.7)
        
    def __len__(self):
        #print len(self.landmarks_frame)
        #return len(self.landmarks_frame)
        if self.datatype == 'train':
            return int(self.trainNum/BATCH_SIZE)+1
        if self.datatype == 'test':
            return int(self.testNum/BATCH_SIZE)+1
    
    
    def __getitem__(self, idx):
#         print(idx)
        landmarks = self.landmarks_frame.get_chunk(BATCH_SIZE).as_matrix().astype('float')
        # landmarks = self.landmarks_frame.ix[idx, 1:].as_matrix().astype('float')
 
        # 采用这个，不错。
        return landmarks
    def refresh(self):
        print("refreshing...")
        self.epoch += 1
        self.landmarks_frame = pd.read_csv(self.csv_file, iterator=True)
        
    def getTrainNum(self):
        return int(self.trainNum/BATCH_SIZE)+1
    
    def getTestNum(self):
        return int(self.testNum/BATCH_SIZE)+1

filename = './trainset.csv'
dataset = FaceLandmarksDataset(filename,'train')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

testname = './testset.csv'
testset = FaceLandmarksDataset(testname,'test')
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=True)

In [3]:
# for batch_data in dataloader:
#     print(shuffle(batch_data[0][:,256]))

In [4]:
class Net(torch.nn.Module):     # 继承 torch 的 Module
    def __init__(self, n_feature):
        super(Net, self).__init__()     # 继承 __init__ 功能
        self.hidden1 = torch.nn.Linear(n_feature, 128)   # 隐藏层线性输出
        self.hidden2 = torch.nn.Linear(128, 64)
        self.hidden3 = torch.nn.Linear(64, 32)
        self.out = torch.nn.Linear(32, 2)       # 输出层线性输出

    def forward(self, x):
        # 正向传播输入值, 神经网络分析出输出值
        x = F.relu(self.hidden1(x))      # 激励函数(隐藏层的线性值)
        x = F.relu(self.hidden2(x)) 
        x = F.relu(self.hidden3(x)) 
        x = self.out(x)                 # 输出值, 但是这个不是预测值, 预测值还需要再另外计算
        x = F.log_softmax(x, dim=1)
        return x
    
    

net = Net(n_feature=128*2) # 几个类别就几个 output

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [5]:
# optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
optimizer = optim.Adam(net.parameters(), lr=0.01, betas=(0.9, 0.999), eps=1e-08, weight_decay=5e-4, amsgrad=False)
criterion = torch.nn.CrossEntropyLoss()  # the target label is NOT an one-hotted

In [6]:
# for batch_data in dataloader:
#     print(batch_data[0][:,256])

In [7]:
best_acc = 0

def train(epoch):
    print('\ntraining Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    i = 0
    for batch_data in dataloader:
        inputs = batch_data[0][:,1:257]
        inputs = inputs.float()
        targets = batch_data[0][:,257]
        targets = targets.long()
  
        if inputs.shape[0] < BATCH_SIZE:
            dataset.refresh()
            
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = net(inputs)
#         outputs = outputs.long()
        loss = criterion(outputs, targets)
        
        # print(targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        
        if i >= dataset.getTrainNum() - 1:
            print('train set:Loss: %.5f | Acc: %.5f%% (%d/%d)'
                % (train_loss/(i+1), 100.*correct/total, correct, total))
        i += 1
    
def test(epoch):
    print('testing Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    i = 0
    for batch_data in testloader:
        inputs = batch_data[0][:,1:257]
        inputs = inputs.float()
        targets = batch_data[0][:,257]
        targets = targets.long()
  
        if inputs.shape[0] < BATCH_SIZE:
            testset.refresh()
            
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        
        if i >= dataset.getTestNum() - 1:
            print('test set:Loss: %.5f | Acc: %.5f%% (%d/%d)'
                % (train_loss/(i+1), 100.*correct/total, correct, total))
        i += 1

In [8]:
for epoch in range(200):
    train(epoch)
    test(epoch)


training Epoch: 0
refreshing...
train set:Loss: 0.68021 | Acc: 56.67778% (14955/26386)
testing Epoch: 0
test set:Loss: 0.68738 | Acc: 55.80357% (3000/5376)

training Epoch: 1
refreshing...
train set:Loss: 0.67747 | Acc: 57.42060% (15151/26386)
testing Epoch: 1
refreshing...
test set:Loss: 0.67636 | Acc: 57.54126% (3033/5271)

training Epoch: 2
refreshing...
train set:Loss: 0.67755 | Acc: 57.38270% (15141/26386)
testing Epoch: 2
refreshing...
test set:Loss: 0.68612 | Acc: 56.53576% (2980/5271)

training Epoch: 3
refreshing...
train set:Loss: 0.67815 | Acc: 57.20458% (15094/26386)
testing Epoch: 3
test set:Loss: 0.67861 | Acc: 56.45461% (3035/5376)

training Epoch: 4
refreshing...
train set:Loss: 0.67531 | Acc: 57.63663% (15208/26386)
testing Epoch: 4
refreshing...
test set:Loss: 0.67520 | Acc: 57.10491% (3010/5271)

training Epoch: 5
refreshing...
train set:Loss: 0.67538 | Acc: 57.58357% (15194/26386)
testing Epoch: 5
refreshing...
test set:Loss: 0.67476 | Acc: 58.37602% (3077/5271)

t

test set:Loss: 0.65281 | Acc: 58.16733% (3066/5271)

training Epoch: 49
refreshing...
train set:Loss: 0.64792 | Acc: 59.06162% (15584/26386)
testing Epoch: 49
refreshing...
test set:Loss: 0.65427 | Acc: 57.99659% (3057/5271)

training Epoch: 50
refreshing...
train set:Loss: 0.64577 | Acc: 59.55431% (15714/26386)
testing Epoch: 50
test set:Loss: 0.65388 | Acc: 58.66815% (3154/5376)

training Epoch: 51
refreshing...
train set:Loss: 0.64798 | Acc: 59.06920% (15586/26386)
testing Epoch: 51
refreshing...
test set:Loss: 0.65346 | Acc: 58.58471% (3088/5271)

training Epoch: 52
refreshing...
train set:Loss: 0.64694 | Acc: 59.08057% (15589/26386)
testing Epoch: 52
refreshing...
test set:Loss: 0.65352 | Acc: 59.55227% (3139/5271)

training Epoch: 53
refreshing...
train set:Loss: 0.64605 | Acc: 59.63390% (15735/26386)
testing Epoch: 53
refreshing...
test set:Loss: 0.65084 | Acc: 58.43293% (3080/5271)

training Epoch: 54
refreshing...
train set:Loss: 0.64458 | Acc: 59.45577% (15688/26386)
testing 

KeyboardInterrupt: 

In [12]:
landmarks_frame = pd.read_csv('./dataset.csv')
# './trainset.csv'

print(landmarks_frame.shape)
# landmarks_frame[:10].to_csv('result1.csv')

(36599, 258)
